해당 파일은 추후 특정 input 이미지에 대해 전체 연예인 embedding vector과의 거리를 계산할 때,<br/>시간 단축을 위해 전체 연예인 embedding vector, 일부 웹툰 embedding vector 관련 정보를 따로 파일로 저장해두는 과정입니다.

저장되는 정보는 크게 2개이며 이는 아래와 같습니다.<br/>
**(1) embedding vector**<br/>
type: np.array<br/>
`연예인` shape: (19708, 512)<br/>
*19708은 crop 결과물 개수<br/>
`웹툰` shape : (352,512)<br/>
*352는 웹툰 image 전체 개수<br/><br/>
**(2) 이미지 경로**<br/>
type: list<br/>
`연예인`<br/>
각 원소의 형태: [연예인 폴더명, 이미지 파일명]<br/>
len: 19708<br/>
*19708은 crop 결과물 개수<br/>
`웹툰`<br/>
각 원소의 형태: [웹툰 폴더명, 웹툰 파일명]<br/>
len: 352<br/>
*352는 웹툰 image 전체 개수<br/><br/>
사용자가 직접 변수값을 입력해줘야 하는 부분은<br/>**0.변수설정**입니다. 

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as nnf
import numpy as np
import os
import matplotlib.pyplot as plt
import math
import cv2
import pickle
from tqdm import tqdm

workers = 0 if os.name == 'nt' else 4
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

#### 0.변수설정 

In [ ]:
# data_path: 연예인 폴더들을 포함하는 상위 폴더
actor_data_path = "/opt/ml/embedding_visualization/data/actor_data"

# actor_embedding_saving_path: embedding vector 변수가 저장되는 파일 위치
actor_embedding_saving_path = "/opt/ml/embedding_visualization/data/actor_embedding_saving.data"

# actor_embedding_info_path: embedding vector에 해당하는 이미지 경로정보가 저장되는 파일 위치
actor_embedding_info_path = "/opt/ml/embedding_visualization/data/actor_embedding_info.data"

# actor_tensorboard_img_path : tensorboard에서 img visualization을 위해 사용하는 img가 저장되는 파일 위치
actor_tensorboard_img_path = "/opt/ml/embedding_visualization/data/actor_tensorboard_img.data"

# webtoon_data_path : 웹툰 폴더들을 포함하는 상위 폴더
webtoon_data_path = "/opt/ml/embedding_visualization/data/webtoon"

# webtoon_embedding_saving_path: embedding vector 변수가 저장되는 파일 위치
webtoon_embedding_saving_path = "/opt/ml/embedding_visualization/data/webtoon_embedding_saving.data"

# webtoon_embedding_info_path: embedding vector에 해당하는 이미지 경로정보가 저장되는 파일 위치
webtoon_embedding_info_path = "/opt/ml/embedding_visualization/data/webtoon_embedding_info.data"

# webtoon_tensorboard_img_path : tensorboard에서 webtoon img visualization을 위해 사용하는 img가 저장되는 파일 위치
webtoon_tensorboard_img_path = "/opt/ml/embedding_visualization/data/webtoon_tensorboard_img.data"

#### 1. MTCNN 모듈 및 InceptionResnetV1 모듈 정의

In [ ]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [ ]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

#### 2. 사진이 없는 폴더 탐색

In [ ]:
# 연예인 사진이 없는 경우 해당 폴더는 crop대상에서 제외
actor_names = []
remove_actor_names = []
for actor_name in os.listdir(actor_data_path):
    file_path = os.path.join(actor_data_path, actor_name)
    if os.path.isdir(file_path):
        if len(os.listdir(file_path)) > 0: # 해당 폴더에 사진이 있는 경우
            actor_names.append(actor_name)
        else:                              # 해당 폴더에 사진이 없는 경우
            remove_actor_names.append(actor_name)

print(f'사진이 존재하는 배우: {len(actor_names)}명') # 818명
print(f'사진이 존재하지 않는 배우(삭제될 배우): {len(remove_actor_names)}명') # 5명

In [ ]:
# 웹툰 사진이 없는 경우 해당 폴더는 crop대상에서 제외
webtoon_names = []
remove_webtoon_names = []
for webtoon_name in os.listdir(webtoon_data_path):
    file_path = os.path.join(webtoon_data_path, webtoon_name)
    if os.path.isdir(file_path):
        if len(os.listdir(file_path)) > 0: # 해당 폴더에 사진이 있는 경우
            webtoon_names.append(webtoon_name)
        else:                              # 해당 폴더에 사진이 없는 경우
            remove_webtoon_names.append(webtoon_name)

print(f'사진이 존재하는 웹툰: {len(webtoon_names)}명') # 9명
print(f'사진이 존재하지 않는 웹툰(삭제될 웹툰): {len(remove_webtoon_names)}명') # 0명

#### 3. Crop을 위한 custom Dataset정의 및 Data로드

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, data_path, select_names):
        self.mtcnn = mtcnn
        self.imgs, self.labels, self.paths, self.idx_to_class = self.make_dataset(data_path, select_names)
        
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img = self.imgs[idx]
        label = self.labels[idx]
        path = self.paths[idx]
        return img, label, path

    def make_dataset(self, data_path, select_names):
        imgs = []
        labels = []
        paths = []
        idx_to_class = {key:name for key, name in enumerate(select_names)}
        for key, name in idx_to_class.items():
            actor_path = os.path.join(data_path, name)
            for img_name in os.listdir(actor_path):
                # img의 차원이 3이 아니거나, 채널의 개수가 3이 아닌 경우 제외
                img_path = os.path.join(actor_path, img_name)
                img = cv2.imread(img_path)
                if type(img) == np.ndarray and len(img.shape) == 3 and img.shape[2] == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    imgs.append(img)
                    labels.append(key)
                    paths.append(img_path)

        return imgs, labels, paths, idx_to_class

In [ ]:
# total data 실행속도: 1m30s 내외
actor_dataset = CustomImageDataset(actor_data_path, actor_names)
webtoon_dataset = CustomImageDataset(webtoon_data_path, webtoon_names)

In [ ]:
print(f'actor crop 대상 이미지 개수: {len(actor_dataset.imgs)}') # 19725
print(f'actor len(labels): {len(actor_dataset.labels)}') # 19725
print(f'actor dataset.idx_to_class: {actor_dataset.idx_to_class}') # 818
print()
print(f'webtoon crop 대상 이미지 개수: {len(webtoon_dataset.imgs)}') # 352
print(f'webtoon len(labels): {len(webtoon_dataset.labels)}') # 352
print(f'webtoon dataset.idx_to_class: {webtoon_dataset.idx_to_class}') # 9

In [ ]:
def collate_fn(x):
    return x[0]

actor_loader = DataLoader(actor_dataset, collate_fn=collate_fn, num_workers=workers, batch_size=1)
webtoon_loader = DataLoader(webtoon_dataset, collate_fn=collate_fn, num_workers=workers, batch_size=1)

#### 4. Crop 수행 및 수행결과 확인

In [ ]:
# 19725장 -> 14m 
# crop결과물 19708개
actor_aligned = [] # tensor 형태의 face crop image
actor_names = [] # list 형태의 actor name
actor_img_names = [] # img_names 형태의 actor file name
actor_tensorboard_img = [] # tensor 형태의 3x50x50 face crop image
for idx, (x, y, path) in tqdm(enumerate(actor_loader)):
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        actor_aligned.append(x_aligned)
        x = x_aligned.unsqueeze(0)
        actor_tensorboard_img.append(nnf.interpolate(x,size=(50,50)).squeeze(0)) 
        actor_names.append(actor_dataset.idx_to_class[y])
        actor_img_names.append(path.split('/')[-1])

In [ ]:
# 352장 -> 5s -> interploate때문에 분 정도로 늘어남!
webtoon_aligned = [] # tensor 형태의 webtoon image
webtoon_names = [] # list 형태의 webtoon name
webtoon_img_names = [] # img_names 형태의 webtoon file name
webtoon_tensorboard_img = [] # tensor 형태의 3x50x50 webtoon image
tf = transforms.ToTensor()
for idx, (x, y, path) in tqdm(enumerate(webtoon_loader)):
    x = tf(x).unsqueeze(0)
    webtoon_aligned.append(nnf.interpolate(x,size=(160,160)).squeeze(0))
    webtoon_tensorboard_img.append(nnf.interpolate(x,size=(50,50)).squeeze(0)) 
    webtoon_names.append(webtoon_dataset.idx_to_class[y])
    webtoon_img_names.append(path.split('/')[-1])

In [ ]:
# cropped face visualization 
row_size = 10
col_size = min(10, math.ceil(len(webtoon_aligned)//row_size))
fig, ax = plt.subplots(row_size, col_size, figsize=(25, 25)) 
print(len(webtoon_aligned))
for i, output in enumerate(webtoon_aligned[:100]): # embedding 되는 웹툰 image
    ax[i//col_size][i%col_size].imshow((output*255).cpu().numpy().astype(np.Uint64).transpose((1,2,0)))

fig.tight_layout()
plt.show()

In [ ]:
# crop된 face image 원본처럼 시각화 
row_size = 10
col_size = min(10, math.ceil(len(actor_aligned)//row_size))
fig, ax = plt.subplots(row_size, col_size, figsize=(25, 25)) 
for i, output in enumerate(actor_aligned[:100]):
    # (MTCNN은 output을 normalize된 채로 output으로 내보냄!)
    ax[i//row_size][i%col_size].imshow((output * 128 + 127.5).cpu().numpy().astype(np.Uint64).transpose((1,2,0)))

fig.tight_layout()
plt.show()

#### 5. Embedding vector 계산을 위한 Dataset정의 및 데이터로드

In [ ]:
class EmbeddingDataset(Dataset):
    def __init__(self, aligned: list, names: list, img_names: list, tensorboard_img: list):
        self.aligned_torch = torch.stack(aligned)
        self.names = names
        self.img_names = img_names
        self.tensorboard_img  = tensorboard_img

    def __len__(self):
        return len(self.aligned_torch)

    def __getitem__(self, idx):
        return self.aligned_torch[idx], self.names[idx], self.img_names[idx], self.tensorboard_img[idx]

In [ ]:
actor_aligned_dataset = EmbeddingDataset(actor_aligned, actor_names, actor_img_names, actor_tensorboard_img)
actor_alinged_loader = DataLoader(actor_aligned_dataset, num_workers=workers, batch_size=50)

In [ ]:
print(f'stack된 aligned shape: {actor_aligned_dataset.aligned_torch.shape}') # torch.Size([19708, 3, 160, 160])
print(f'len(aligned_dataset): {len(actor_aligned_dataset)}') # 19708
print(f'len(alinged_loader): {len(actor_alinged_loader)}\n') # 395

print(f'len(names): {len(actor_aligned_dataset.names)}') # 19708
print(f'names[0]: {actor_aligned_dataset.names[0]}\n') # 곽지민

print(f'len(img_names): {len(actor_aligned_dataset.img_names)}') # 19708
print(f'img_names[0]: {actor_aligned_dataset.img_names[0]}\n') # 곽지민2.jpg

print(f'len(tensorboard_img): {len(actor_tensorboard_img)}') # 19708
print(f'tensorboard_img[0].shape: {actor_tensorboard_img[0].shape}') # torch.Size([3, 50, 50])

In [ ]:
webtoon_aligned_dataset = EmbeddingDataset(webtoon_aligned, webtoon_names, webtoon_img_names, webtoon_tensorboard_img)
webtoon_alinged_loader = DataLoader(webtoon_aligned_dataset, num_workers=workers, batch_size=50)

In [ ]:
print(f'stack된 aligned shape: {webtoon_aligned_dataset.aligned_torch.shape}') # torch.Size([352, 3, 160, 160])
print(f'len(aligned_dataset): {len(webtoon_aligned_dataset)}') # 352
print(f'len(alinged_loader): {len(webtoon_alinged_loader)}\n') # 8

print(f'len(names): {len(webtoon_aligned_dataset.names)}') # 352
print(f'names[0]: {webtoon_aligned_dataset.names[0]}\n') # Unknown1

print(f'len(img_names): {len(webtoon_aligned_dataset.img_names)}') # 352
print(f'img_names[0]: {webtoon_aligned_dataset.img_names[0]}\n') # 4422_pp_0.jpg

print(f'len(tensorboard_img): {len(webtoon_tensorboard_img)}') # 352
print(f'tensorboard_img[0].shape: {webtoon_tensorboard_img[0].shape}') # torch.Size([3, 50, 50])

#### 6. InceptionResnetV1 수행

In [ ]:
# 19708장 -> 16.3s
actor_saving_embeddings = []
actor_saving_names = []
actor_saving_img_names = []
actor_saving_tensorboard_img = torch.stack(actor_tensorboard_img).to(device)
for x, name, img_name, tensorboard_img in actor_alinged_loader:
    embedding = resnet(x.to(device))
    actor_saving_embeddings.extend(embedding.detach().cpu().tolist())
    actor_saving_names.extend(name)
    actor_saving_img_names.extend(img_name)

actor_saving_img_infos = list(map(list, zip(actor_saving_names, actor_saving_img_names)))

In [ ]:
print(f'actor embedding shape: {np.array(actor_saving_embeddings).shape}') # (19708, 512)
print(f'actor len(saving_img_infos): {len(actor_saving_names)}') # 19708
print(f'actor saving_img_infos[0]: {actor_saving_img_infos[0]}') # ['곽지민', '곽지민2.jpg'] example

In [ ]:
# 392장 -> 
webtoon_saving_embeddings = []
webtoon_saving_names = []
webtoon_saving_img_names = []
webtoon_saving_tensorboard_img = torch.stack(webtoon_tensorboard_img).to(device)
for x, name, img_name, tensorboard_img in webtoon_alinged_loader:
    embedding = resnet(x.to(device))
    webtoon_saving_embeddings.extend(embedding.detach().cpu().tolist())
    webtoon_saving_names.extend(name)
    webtoon_saving_img_names.extend(img_name)

webtoon_saving_img_infos = list(map(list, zip(webtoon_saving_names, webtoon_saving_img_names)))

In [ ]:
print(f'webtoon embedding shape: {np.array(webtoon_saving_embeddings).shape}') # (352, 512)
print(f'webtoon len(saving_img_infos): {len(webtoon_saving_names)}') # 352
print(f'webtoon saving_img_infos[0]: {webtoon_saving_img_infos[0]}') # ['Unknown1', '4422_pp_0.jpg'] example

#### 7. pickle 저장 수행

##### actor 관련 저장

In [ ]:
# actor_saving_embeddings 저장
with open(actor_embedding_saving_path, 'wb') as f:
    pickle.dump(actor_saving_embeddings, f)
# actor_saving_img_infos 저장
with open(actor_embedding_info_path, 'wb') as f:
    pickle.dump(actor_saving_img_infos, f)
# actor_saving_tensorboard_img 저장
with open(actor_tensorboard_img_path, 'wb') as f:
    pickle.dump(actor_saving_tensorboard_img, f)

##### webtoon 관련 저장

In [ ]:
# webtoon_saving_embeddings 저장
with open(webtoon_embedding_saving_path, 'wb') as f:
    pickle.dump(webtoon_saving_embeddings, f)
# webtoon_saving_img_infos 저장
with open(webtoon_embedding_info_path, 'wb') as f:
    pickle.dump(webtoon_saving_img_infos, f)
# webtoon_saving_tensorboard_img 저장
with open(webtoon_tensorboard_img_path, 'wb') as f:
    pickle.dump(webtoon_saving_tensorboard_img, f)